In [1]:
import time
import dask
from dask.distributed import Client
import os
import subprocess
from pathlib import Path
import pprint
from typing import List

In [2]:
%%bash
python --version 

Python 3.8.10


In [3]:
pp = pprint.PrettyPrinter()

In [4]:
def task(cmd: List[str]):
    out = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return out

In [5]:
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")
# create a client
client = Client(scheduler_file=scheduler_file)

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
client

<Client: 'tcp://10.249.44.97:8786' processes=500 threads=128000, memory=232.70 TiB>

## Testing Job

In [7]:
# submit a testing task
num_tasks = 10_000
futures = []
for i in range(num_tasks):
    futures.append(client.submit(task, ["ls", "-l"]))

# wait for the task to finish
futures = client.gather(futures)

In [8]:
print(len(futures))
pp.pprint(futures[1].stdout)

10000
('Dask_Example-Container.ipynb\n'
 'Dask_Example.ipynb\n'
 '__pycache__\n'
 'create_ray_jobs.py\n'
 'dask_client.py\n'
 'detectors_odd.csv\n'
 'full_chain_odd.py\n'
 'get_py_cmd.sh\n'
 'jobs.txt\n'
 'launch-dask-shifter-cpu.sh\n'
 'launch-dask-shifter-gpu.sh\n'
 'odd-sp-config.json\n'
 'prepare.py\n'
 'run_acts_in_shifter.sh\n'
 'run_generic.sh\n'
 'stderr.txt\n'
 'stdout.txt\n'
 'submit.sh\n')


## ACTS jobs

In [7]:
num_jobs = 500
num_evts = 1000
outdir = "dask_v5_p500"

# cmd = ["./run_acts_in_shifter.sh", str(num_evts), outdir]
cmd = f"./run_acts_in_shifter.sh {num_evts} {outdir}"

In [8]:
futures = []
for i in range(num_jobs):
    seed = 756323 + i
    full_cmd = cmd + f" {seed}"
    futures.append(client.submit(task, full_cmd))

# wait for the task to finish
futures = client.gather(futures)